In [1]:
from IPython.display import Image
import math

- GQA 允许使用比 query head 数量更少的 key/value head，以节省计算和内存
    - N_q = 4, N_kv = 2
        - llama3: `N_q = 32, N_kv = 8`
    - key 和 value head 会被复制以匹配 query head 的数量。
        - 在 GQA 中，查询头被分成若干组，每组查询头共享同一组键 (Key) 和值 (Value) 头。
        - N_q 必须是 N_kv 的整数倍。例如，如果你有 8 个查询头 (N_q=8) 和 2 个键/值头 (N_kv=2)，那么每 4 个查询头会共享同一组键/值头。

### padding vs. packing

In [2]:
Image(url='./imgs/packing_padding.jpeg', width=400)


- 左侧是传统的padding做法，右侧是packing，其中红色部分代表pad token，黄色部分代表sep token。
    - 都是整理成 batch tensor
    - 为了区分不同的训练示例，我们在不同示例之间加上一个分割标记sep token，
- **注意力窗口不允许跨示例**。
    - padding：传统的全局下三角矩阵。
    - packing：这个注意力模式叫块对角矩阵（BlockDiagonalMask）【本质上是在示例内的下三角矩阵】，
        - 由此，就消除了对pad token的需要，所以开源大模型刚问世的时候（2023-3那阵子），存在很多base model放出来的tokenizer并没有pad token，比如llama-base。
        - 需要注意，packing时示例3可能会被截断，这个行为在预训练时是可以接受的。注意，这个时候的学习模式非常的简单，就是next token prediction。
    

### sdpa

- https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.scaled_dot_product_attention.html

In [2]:
import torch
import torch.nn.functional as F
from torch.nn.attention import SDPBackend, sdpa_kernel

In [3]:
seed = 42
torch.manual_seed(seed)

In [3]:
# query: (B, N_q, L, D_k)
# key: (B, N_kv, S, D_k)
# value: (B, N_kv, S, D_v)
query = torch.rand(2, 4, 3, 2, dtype=torch.float16, device="cuda")
key = torch.rand(2, 4, 3, 2, dtype=torch.float16, device="cuda")
value = torch.rand(2, 4, 3, 2, dtype=torch.float16, device="cuda")

output = F.scaled_dot_product_attention(query, key, value, is_causal=True)

In [10]:
# output

In [5]:
L, S = query.size(-2), key.size(-2)
L, S, query.size(-1)

(3, 3, 2)

In [6]:
scale_factor = 1 / math.sqrt(query.size(-1))
scale_factor

0.7071067811865475

In [7]:
attn_bias = torch.zeros(L, S, dtype=query.dtype, device=query.device)
temp_mask = torch.ones(L, S, dtype=torch.bool, device=query.device).tril(diagonal=0)
attn_bias.masked_fill_(temp_mask.logical_not(), float("-inf"))
attn_bias.to(query.dtype)
temp_mask, attn_bias

(tensor([[ True, False, False],
         [ True,  True, False],
         [ True,  True,  True]], device='cuda:0'),
 tensor([[0., -inf, -inf],
         [0., 0., -inf],
         [0., 0., 0.]], device='cuda:0', dtype=torch.float16))

In [11]:
attn_weight = query @ key.transpose(-2, -1) * scale_factor

# 通过加一个上三角为 -inf 的下三角为 0，然后 softmax 实现 causal mask
attn_weight += attn_bias
attn_weight = torch.softmax(attn_weight, dim=-1)

attn_weight = torch.dropout(attn_weight, 0., train=True)
output1 = attn_weight @ value

### enbale gqa

In [8]:
# query: (B, N_q, L, D_k)
# key: (B, N_kv, S, D_k)
# value: (B, N_kv, S, D_v)
query = torch.rand(2, 4, 3, 2, dtype=torch.float16, device="cuda")
key = torch.rand(2, 2, 3, 2, dtype=torch.float16, device="cuda")
value = torch.rand(2, 2, 3, 2, dtype=torch.float16, device="cuda")

output = F.scaled_dot_product_attention(query, key, value, enable_gqa=True)